# Collapsing Things in netCDF4
# May 19, 2017

In [1]:
import numpy as np
import numpy.ma as ma
import netCDF4 as nc
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
day1u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141218-20141218.nc')
day2u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141219-20141219.nc')
day3u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141220-20141220.nc')
day4u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141221-20141221.nc')
day5u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141222-20141222.nc')
day6u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141223-20141223.nc')
day7u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141224-20141224.nc')                      
day8u = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_U_20141225-20141225.nc')

day1v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141218-20141218.nc')
day2v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141219-20141219.nc')
day3v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141220-20141220.nc')
day4v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141221-20141221.nc')
day5v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141222-20141222.nc')
day6v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141223-20141223.nc')
day7v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141224-20141224.nc')
day8v = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_V_20141225-20141225.nc')

day1w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141218-20141218.nc')
day2w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141219-20141219.nc')
day3w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141220-20141220.nc')
day4w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141221-20141221.nc')
day5w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141222-20141222.nc')
day6w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141223-20141223.nc')
day7w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141224-20141224.nc')
day8w = nc.Dataset('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h_20141218_20141225_grid_W_20141225-20141225.nc')

u_files = [day1u,day2u,day3u,day4u,day5u,day6u,day7u,day8u]
v_files = [day1v,day2v,day3v,day4v,day5v,day6v,day7v,day8v]
w_files = [day1w,day2w,day3w,day4w,day5w,day6w,day7w,day8w]

mesh_mask = nc.Dataset('/home/vdo/MEOPAR/NEMO-forcing/grid/mesh_mask201702.nc')
delta_x = mesh_mask['e1u']
delta_y = mesh_mask['e2v']
delta_z = mesh_mask['e3w_1d']
t_mask = mesh_mask['tmask']
u_mask = mesh_mask['umask']
v_mask = mesh_mask['vmask']                 

In [53]:
def pcourantu(files,meshmask):
    """Given a list of U files and a mask mask, returns an array with the unscaled Counrant numbers.
    
    :arg files: list of U files
    
    :arg meshmask: mesh mask 
    :type mashmask: :py:class:'netCDF4.Dataset'
    
    :returns: Numpy MaskedArray with unscaled Courant numbers.
    :rtype: :py:class: 'numpy.ma.core.MaskedArray'
    """
    
    ubdx = np.zeros((40,898,398))
    delta_x = meshmask['e1u']
    for file in files:
        for n in range(0,24):
            u = np.abs(file['vozocrtx'][n,:,:,:] / delta_x)
            ubdx = np.maximum(u,ubdx)     #taking maximum over time
    new_u = np.zeros((898,398))
    for m in range(0,40):
        u = ubdx[m,:,:]
        new_u = np.maximum(u,new_u)    #taking maximum over depth
    
    return new_u

In [67]:
type(mesh_mask)

netCDF4._netCDF4.Dataset

In [65]:
pcourantu(u_files,mesh_mask)

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]],
       fill_value = 1e+20)

In [55]:
def pcourantv(files,meshmask):
    """Given a list of V files and a mesh mask, returns an array with the unscaled Courant numbers.
    
    :arg files: list of U files
    
    :arg meshmask: mesh mask 
    :type mashmask: :py:class:'netCDF4.Dataset'
    
    :returns: Numpy MaskedArray with unscaled Courant numbers.
    :rtype: :py:class: 'numpy.ma.core.MaskedArray' 
    """

    vbdx = np.zeros((40,898,398))
    delta_y = meshmask['e2v']
    for file in files:
        for n in range(0,24):
            v = np.abs(file['vomecrty'][n,:,:,:] / delta_y)
            vbdx = np.maximum(v,vbdx)   #taking maximum over time

    new_v = np.zeros((898,398))
    for m in range(0,40):
        v = vbdx[m,:,:]
        new_v = np.maximum(v,new_v)  #taking maximum over depth

    return new_v

In [56]:
pcourantv(v_files,mesh_mask)

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]],
       fill_value = 1e+20)

In [61]:
def pcounrantw(files,meshmask):
    """Given a list of W files and a mesh mask, returns an array with the unscaled Courant numbers.
    
    :arg files: list of U files
    
    :arg meshmask: mesh mask 
    :type mashmask: :py:class:'netCDF4.Dataset'
    
    :returns: Numpy MaskedArray with unscaled Courant numbers.
    :rtype: :py:class: 'numpy.ma.core.MaskedArray'
    """

    wbdx = np.zeros((40,898,398))
    delta_z = meshmask['e3w_1d']
    new_z1 = np.expand_dims(delta_z[:],axis=2)
    new_z2 = np.swapaxes(new_z,0,1)
    ones = np.ones((40,898,398))
    new_z3 = ones*new_z2
    
    for file in files:
        for n in range(0,24):
            w = np.abs(file['vovecrtz'][n,:,:,:] / new_z3)   
            wbdx = np.maximum(w,wbdx)   #taking maximum over time

    new_w = np.zeros((898,398))
    for m in range(0,40):
        w = wbdx[m,:,:]
        new_w = np.maximum(w,new_w)  #taking maximum over depth
    
    return new_w

In [62]:
pcounrantw(w_files,mesh_mask)

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]],
       fill_value = 1e+20)